# BLOB DETECTION

BLOB stands for **Binary Large Object**

Informally a blob is a region of an image in which some properties like intensity or color are approximately constant.



Importing cv2 and numpy libary

In [1]:
import cv2
import numpy as np

# Preproccessing
## 1. Choosing the color of blob
* loading the live video through webcam using cv2.VideoCapture()
* Reading the frame using video.read()
* select a region in a frame of your color choice using cv2.selectROI()
![](./images/roi_blob.png)

## 2. Converting to HSV and finding bounding values for mask
* Converting the frame to HSV format using cv2.cvtColor(_,cv2.COLOR_BGR2HSV)
    * Why converting to HSV?  
        Since we are using the web cam the intensity and illumination of consecutive frame does not remain same.  
        Hence to find the color in range instead of particular color. HSV format is useful as H value denotes specific color and S, V can be used for illumination and intensity.  
        ![](./images/hsv_blob.png)
* Extracting the select region from roi using index slicing
* Calculating the median H,S,V values from roi
* Initializing the lower and upper bound for mask

In [2]:
#Start the webcam in your device
#'0' is a special argument to open the webcam
video = cv2.VideoCapture(0)

#An infinite loop to get each frame of the video
while(video.isOpened()):
    _, frame = video.read()     #Read the video frame
    cv2.imshow("Image",frame)   #Show each frame of the video

    if cv2.waitKey(10) == ord('q'):   #If in interval of 10ms 'q' is pressed then execute below
        bbox = cv2.selectROI(frame)   #Select the ROI(Region Of Interest)
        # selectROI returns a tuple of x,y,h,w
        x = bbox[0] # x cordinate of leftmost point in ROI
        y = bbox[1] # y cordinate of leftmost point in ROI
        w = bbox[2] # width of ROI
        h = bbox[3] # height of ROI 
        
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)   #Convert the BGR format to HSV format

        obj_img = hsv[ y:y+h , x:x+w ]    #slice the ROI from image

        h, s, v = np.median(obj_img[:,:,0]), np.median(obj_img[:,:,1]), np.median(obj_img[:,:,2])     #Taking median of the hsv values in the ROI

        lower = np.array([h-5, max(0,s-50),max(0,v-50)])        #The lowerbound values for hsv
        upper = np.array([h+5, min(s+50,255),min(v+50,255)])    #The upperbound values for hsv
        print(lower,upper)
        cv2.destroyAllWindows()
        video.release()
        
        break

[ WARN:0] global ../modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


[171. 113. 107.] [181. 213. 207.]


# Detecting the blob
## Constructing mask for detecting blob
* Read the frame from video
* Convert from BGR to HSV
* Make a mask using cv2.inRange() by passing lower and upper bounds calculated earlier
    * What is a mask?  
    A mask is a binary image consisting of zero and non-zero values. If a mask is applied to another image of the same size, all pixels which are zero in the mask are set to zero in the output image. All others remain unchanged. 
   
  ![](./images/not_blurredmask.png)
* Blur the mask to remove the noise using cv2.medianBlur()  

![](./images/blurred_mask.png)
* Placing mask over frame to finded colored mask using cv2.bitwise_and()

![](./images/colored_mask.png)


## Drawing the blob
* Find the contour from the generated mask using cv2.findContours()
    * What is contour?  
    Contours can be explained simply as a curve joining all the continuous points (along the boundary), having same color or intensity. The contours are a useful tool for shape analysis and object detection and recognition

    ![](./images/multiple_contours.png) 
    ![](./images/blob.png)
    
* Find the contour having the maximum area using cv2.contourArea()
* Draw the contour on the frame using cv2.drawContours()

In [7]:
video = cv2.VideoCapture(0)
while(video.isOpened()):
    _, frame = video.read()

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) 

    masked = cv2.inRange(hsv, lower, upper) #Masking the frames in the video for the given range

    blur = cv2.medianBlur(masked, 5)        #Applying meadin Blur on the frame

    blob_mask = cv2.bitwise_and(frame,frame,mask=blur)  #Using bitwise-and to show only the ROI object

    cv2.imshow("blob_mask",blob_mask)                   #Showing the output of bitwise_and

    contours, _ = cv2.findContours(blur, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)    #Detect the contours and store them in a list

    idx, current_max, counter = 0, 0, 0     #idx = index of contour with maximum area, current_max = Contour with max ares, counter = temporary variable

    for n in contours:                      #To find the index of the contour with maximum area
        area = cv2.contourArea(n)
        if area > current_max:
            current_max = area
            idx = counter
        counter += 1

    cv2.drawContours(frame, contours, idx, (0,0,255),2)     #Draw the contours on the region
    cv2.imshow("Output",frame)                              #Show the output with the contours

    if cv2.waitKey(10) == ord('x'):                         #When 'x' is pressed at an interval of 10 ms, then close all windows
        cv2.destroyAllWindows()
        video.release()
        break

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


[ WARN:0] global ../modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [ ]:
#complete code
import numpy as np 
import cv2

video = cv2.VideoCapture(0)

while(video.isOpened()):
    _, frame = video.read()
    cv2.imshow("Image",frame)

    if cv2.waitKey(10) == ord('q'):
        bbox = cv2.selectROI(frame)
        
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

        obj_img = hsv[bbox[1]:bbox[1]+bbox[3],bbox[0]:bbox[0]+bbox[2]]

        h, s, v = np.median(obj_img[:,:,0]), np.median(obj_img[:,:,1]), np.median(obj_img[:,:,2])

        lower = np.array([h-5, max(0,s-50),max(0,v-50)])
        upper = np.array([h+5, min(s+50,255),min(v+50,255)])
        print(lower,upper)
        break


while(video.isOpened()):
    _, frame = video.read()

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    masked = cv2.inRange(hsv, lower, upper)

    blur = cv2.medianBlur(masked, 5)

    blob_mask = cv2.bitwise_and(frame,frame,mask=blur)

    cv2.imshow("blob_mask",blob_mask)

    contours, _ = cv2.findContours(blur, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    idx, current_max, counter = 0, 0, 0

    for n in contours:
        area = cv2.contourArea(n)
        if area > current_max:
            current_max = area
            idx = counter
        counter += 1

    cv2.drawContours(frame, contours, idx, (0,255,255),2)
    cv2.imshow("Output",frame)

    if cv2.waitKey(10) == ord('x'):
        cv2.destroyAllWindows()
        video.release()
        break


In [ ]:
#complete code for not having webcam
import numpy as np 
import cv2

video = cv2.VideoCapture('./video/Blob_detection_video.mp4')


_, frame = video.read()
cv2.imshow("Image",frame)

bbox = cv2.selectROI(frame)

hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

obj_img = hsv[bbox[1]:bbox[1]+bbox[3],bbox[0]:bbox[0]+bbox[2]]

h, s, v = np.median(obj_img[:,:,0]), np.median(obj_img[:,:,1]), np.median(obj_img[:,:,2])

lower = np.array([h-5, max(0,s-50),max(0,v-50)])
upper = np.array([h+5, min(s+50,255),min(v+50,255)])
print(lower,upper)



while(video.isOpened()):
    _, frame = video.read()

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    masked = cv2.inRange(hsv, lower, upper)

    blur = cv2.medianBlur(masked, 5)

    blob_mask = cv2.bitwise_and(frame,frame,mask=blur)

    cv2.imshow("blob_mask",blob_mask)

    contours, _ = cv2.findContours(blur, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    idx, current_max, counter = 0, 0, 0

    for n in contours:
        area = cv2.contourArea(n)
        if area > current_max:
            current_max = area
            idx = counter
        counter += 1

    cv2.drawContours(frame, contours, idx, (0,255,255),2)
    cv2.imshow("Output",frame)

    if cv2.waitKey(10) == ord('x'):
        cv2.destroyAllWindows()
        video.release()
        break

To learn more interesting stuffs of computer vision using opencv [click here](https://docs.opencv.org/master/d9/df8/tutorial_root.html)